## An Alternate Method for Data Acquisition

I noticed some issues with the previous dataset. Notably, the volume column contained many zero values. With enough incorrect values from the source data, errors can propogate to future pipeline steps. To correct this, I will instead use [The Investors Exchange](https://iextrading.com/), which provides free historical stock data. This data is more accurate and reliable than the Kaggle repo sourced from Yahoo Finance.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

To acquire the data, an API key from [Alpaca](https://alpaca.markets/) is required. Alpaca provides an easy to use API for stock trading, market data acquisition, and backtesting. Some features require an authorized account, but for my needs a basic account will work. To utilize the API  you must first generate two keys, a key ID and a secret key. 

In [2]:
import alpaca_trade_api as tradeapi

key_id = None
secret_key = None
with open('../files/private/alpakey') as key_file:
    keys = key_file.readlines()
    key_id = keys[0].strip()
    secret_key = keys[1].strip()
    
api_url = "https://paper-api.alpaca.markets"

alpaca = tradeapi.REST(key_id, secret_key, api_url, api_version='v2')

'top_symbols.json' contains a list of the 500 most frequently traded stock symbols, compiled by scraping The Investors Exchange's website.

In [3]:
import json

symbols = []
with open('../files/public/top_symbols.json') as top_symbols:
    symbols = json.load(top_symbols)

The API restricts users to 100 symbols per call. You can specify the timeperiod between each data point, the number of data points, and the starting or ending point. The results can be returned as a DataFrame for convenience. To train the model, the last 1000 days of data will be used. This data dates back to the middle of 2016.

In [4]:
stocks_current = []

for i in tqdm(range(0, len(symbols), 100)):
    stocks_current.append(alpaca.get_barset(symbols[i:i+100], '1D', 1000).df.stack(level=0).reset_index())

For backtesting the model, unseen data will need to be acquired. However, due to fluctuations in the market, not every year makes for balanced data. To avoid this, I will backtest using three sets of data. 

##### Backtest Set 1: 2008
    
The market crash of 2008 was one of the worst on record. I'll choose this year to test the model's performance in the face of a recession.
    
##### Backtest Set 2: 2011

2011 was neither a good year or a bad year for the market. There was little difference between stock prices in January and December. I'll choose this year to test the model's performance in flat years.

##### Backtest Set 3: 2013

By 2013, the market had rebounded from the 2008 recession. Throughout the year, stock growth never stopped. I'll choose this year to test the model's performance in a successful market.

In [5]:
stocks_2008 = []
for i in tqdm(range(0, len(symbols), 100)):
    stocks_2008.append(alpaca.get_barset(symbols[i:i+100], '1D', 500, end='2009-01-01T00:00:00-00:00').df.stack(level=0).reset_index())
    
stocks_2011 = []
for i in tqdm(range(0, len(symbols), 100)):
    stocks_2011.append(alpaca.get_barset(symbols[i:i+100], '1D', 500, end='2012-01-01T00:00:00-00:00').df.stack(level=0).reset_index())
    
stocks_2013 = []
for i in tqdm(range(0, len(symbols), 100)):
    stocks_2013.append(alpaca.get_barset(symbols[i:i+100], '1D', 500, end='2014-01-01T00:00:00-00:00').df.stack(level=0).reset_index())    

After the stocks have been retrieved, I concatenate them into a single DataFrame.

In [6]:
stocks_current = pd.concat(stocks_current, ignore_index=True)
stocks_2008 = pd.concat(stocks_2008, ignore_index=True)
stocks_2011 = pd.concat(stocks_2011, ignore_index=True)
stocks_2013 = pd.concat(stocks_2013, ignore_index=True)

Some ETFs (Exchange Traded Funds) track multiple stocks and can be used to approximate the market as a whole. The SPY ETF tracks the S&P 500, a collection of stocks listed on the US markets. By combining stock data with overall market data, better predictions can be made that take into account market ups and downs. Other ETFs may work just as well, such as DIA (Dow Jones Industrial Average) or VUG (Vanguard). In my tests, they all performed similarly.

In [7]:
market_current = alpaca.get_barset('SPY', '1D', 1000).df.stack(level=0).reset_index()
market_2008 = alpaca.get_barset('SPY', '1D', 1000, end='2009-01-01T00:00:00-00:00').df.stack(level=0).reset_index()
market_2011 = alpaca.get_barset('SPY', '1D', 1000, end='2012-01-01T00:00:00-00:00').df.stack(level=0).reset_index()
market_2013 = alpaca.get_barset('SPY', '1D', 1000, end='2014-01-01T00:00:00-00:00').df.stack(level=0).reset_index()

In [8]:
cols = ['date', 'symbol', 'close', 'high', 'low', 'open', 'volume']

def rename_columns(stocks, market):
    stocks.columns = cols
    market.columns = cols

rename_columns(stocks_current, market_current)
rename_columns(stocks_2008, market_2008)
rename_columns(stocks_2011, market_2011)
rename_columns(stocks_2013, market_2013)

In [9]:
def convert_dates(stocks, market):
    stocks.date = pd.to_datetime(stocks.date, utc=True)
    market.date = pd.to_datetime(market.date, utc=True)

convert_dates(stocks_current, market_current)
convert_dates(stocks_2008, market_2008)
convert_dates(stocks_2011, market_2011)
convert_dates(stocks_2013, market_2013)

The stock DataFrames are merged with the market DataFrame by date.

In [10]:
stocks_current = stocks_current.merge(market_current, on='date', suffixes=['', '_market'])
stocks_current.drop('symbol_market', axis=1, inplace=True)

stocks_2008 = stocks_2008.merge(market_2008, on='date', suffixes=['', '_market'])
stocks_2008.drop('symbol_market', axis=1, inplace=True)

stocks_2011 = stocks_2011.merge(market_2011, on='date', suffixes=['', '_market'])
stocks_2011.drop('symbol_market', axis=1, inplace=True)

stocks_2013 = stocks_2013.merge(market_2013, on='date', suffixes=['', '_market'])
stocks_2013.drop('symbol_market', axis=1, inplace=True)

In [11]:
stocks_current.set_index('date', inplace=True)
stocks_2008.set_index('date', inplace=True)
stocks_2011.set_index('date', inplace=True)
stocks_2013.set_index('date', inplace=True)

stocks_current.drop_duplicates(inplace=True)
stocks_2008.drop_duplicates(inplace=True)
stocks_2011.drop_duplicates(inplace=True)
stocks_2013.drop_duplicates(inplace=True)

The finalized data is saved for future use. This step may be skipped in future iterations once you are satisfied with the size and quality of the dataset.

In [13]:
stocks_2011[stocks_2011.symbol == 'AAPL']

,symbol,close,high,low,open,volume,close_market,high_market,low_market,open_market,volume_market
date,,,,,,,,,,,
2010-01-08 05:00:00+00:00,AAPL,30.27,30.286,29.866,30.043,1.04222e+08,114.57,114.61,113.660,113.88,95322019
2010-01-11 05:00:00+00:00,AAPL,30.01,30.429,29.779,30.414,1.11354e+08,114.73,115.13,114.235,115.07,89164666
2010-01-12 05:00:00+00:00,AAPL,29.676,29.967,29.489,29.884,1.29701e+08,113.66,114.21,113.220,113.96,136421030
2010-01-13 05:00:00+00:00,AAPL,30.057,30.133,29.157,29.696,1.45123e+08,114.64,114.94,113.370,113.95,142997302
2010-01-14 05:00:00+00:00,AAPL,29.917,30.066,29.86,30.016,9.83561e+07,114.92,115.14,114.420,114.49,99218631
...,...,...,...,...,...,...,...,...,...,...,...
2011-12-23 05:00:00+00:00,AAPL,8.231,8.236,8.153,8.157,4.50485e+08,126.38,126.43,125.410,125.68,64187669
2011-12-27 05:00:00+00:00,AAPL,8.297,8.349,8.224,8.226,4.16494e+08,126.46,126.82,126.060,126.17,62071310
2011-12-28 05:00:00+00:00,AAPL,57.539,57.86,8.253,8.303,2.26845e+08,124.96,126.53,124.730,126.51,102787347


In [12]:
stocks_current.to_hdf('../data/raw/market_stocks.h5', key='stocks_current')
stocks_2008.to_hdf('../data/raw/market_stocks.h5', key='stocks_2008')
stocks_2011.to_hdf('../data/raw/market_stocks.h5', key='stocks_2011')
stocks_2013.to_hdf('../data/raw/market_stocks.h5', key='stocks_2013')

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2530: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['symbol', 'close', 'high', 'low', 'open', 'volume']]

  pytables.to_hdf(path_or_buf, key, self, **kwargs)
/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2530: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->['symbol', 'close', 'high', 'low', 'open', 'volume']]

  pytables.to_hdf(path_or_buf, key, self, **kwargs)
